In [2]:
import visa
from lantz import Feat
from lantz.core import mfeats
from lantz import MessageBasedDriver
from lantz.core import log
from logging import DEBUG

import numpy as np
import matplotlib.pyplot as plt


from lantz import Feat
from lantz.core import mfeats
from lantz import MessageBasedDriver
from lantz.core import log
from logging import DEBUG
import numpy as np
import time


class OsciloscopioTektronix1002B(MessageBasedDriver):

    MANUFACTURER_ID = '0x0699'
    MODEL_CODE = '0x0346'

    @Feat()
    def idn(self):
        return self.query('*IDN?')

    timebase = mfeats.QuantityFeat("HOR:MAI:SCA?","HOR:MAI:SCA {}", units = 's',limits=(0,10000))
    
    voltagescale = mfeats.QuantityFeat("CH1:SCA?","CH1:SCA {}", units = 'V',limits=(0,10))

    def pantalla(self):
        self.write('DAT:ENC RPB')
        self.write('DAT: WID 1')
        data = self.resource.query_binary_values('CURV?', datatype='B',  is_big_endian=True)
        time = np.arange(len(data))
        return time, data

    def escaleo(self):
        #self.inst.write('DAT:ENC ASCIi')
        #self.inst.write('DAT: WID 1')
        print(self.query('WFMP?'))

        #xze, xin, yze, ymu, yoff = self.inst.query_ascii_values('WFMPRE:XZE?;XIN?;YZE?;YMU?;YOFF?;')
        # return xze, xin, yze, ymu, yoff

    def escaleo2(self):
        self.write('DAT:ENC RPB')
        self.write('DAT: WID 1')
        xze, xin, yze, ymu, yoff = self.resource.query_ascii_values(
            'WFMP:XZE?;XIN?;YZE?;YMU?;YOF?;', separator=";")
        return xze, xin, yze, ymu, yoff


ModuleNotFoundError: No module named 'visa'

In [ ]:
class Generador(MessageBasedDriver):

    MANUFACTURER_ID = '0x0699'
    MODEL_CODE = '0x0346'

    @Feat()
    def idn(self):
        return self.query('*IDN?')
    
    @Feat(units = 'Hz',limits=(0,10000))
    def freq(self):
        self.log
        return float(self.query("SOUR1:FREQ:FIX?"))
        
    @freq.setter
    def freq(self, hertz):
        self.write("SOUR1:FREQ:FIX {}".format(hertz))
    
    freqs = mfeats.QuantityFeat("SOUR1:FREQ:FIX?","SOUR1:FREQ:FIX {}", units = 'Hz',limits=(0,10000))
        
    def set_amp(self, volts):
        self.write("SOUR1:VOLT:AMPL {}".format(volts))
   
    def waveform_shape(self, channel = 1, shape = 'SIN'):
        # Formas posibles: SIN, SQU, PULS, RAMP
        # PRNoise, DC|SINC|GAUSsian|LORentz|ERISe|EDECay|
        # HAVersine
        self.write('SOUR{}:FUNC:SHAPE {}'.format(channel, shape))
        
    def waveform_phase(self, radians, channel = 1):
        self.write('SOUR{}:PHAS {})'.format(channel, radians))
    
    def close_instrument(self):
        # Close VISA session (Close instrument connection)
        print("Killing")
        self.close()
        print("VISA session closed!")
    
    def freq_sweep(self, freq_inicial, freq_final, step, stop = 1, channel = 1):
        # Hace un barrido de frecuencias desde freq_inicial hasta freq_final
        # con un step. Entre cada cambio de frecuencia dejamos 1 segundo de
        # stop.
        frecuencias = np.arange(freq_inicial, freq_final, step)
        for elemento in frecuencias:
            self.write('SOUR{}:FREQ:FIX {}'.format(channel, elemento))
            time.sleep(stop)
            

#with AFG3021b('USB0::0x0699::0x0346::C036492::INSTR') as inst:
 #   print(inst.idn)

In [ ]:
gen = Generador('USB0::0x0699::0x0346::C034198::INSTR')
gen.initialize()
osci = OsciloscopioTektronix1002B('USB0::0x0699::0x0363::C065092::INSTR')
osci.initialize()

In [ ]:
gen.freqs = 10 #tiene el problema, de que el query ademas de hacer un write hace un read.

In [ ]:
time, data = osci.pantalla()
xze, xin, yze, ymu, yoff = osci.escaleo2()
data = np.array([float(i) for i in data])
time = np.array([float(i) for i in time])
data = -yoff*ymu + data*ymu 
time = xze + time * xin

In [ ]:
plt.plot(time,data)


In [ ]:
#Dependiendo que estamos levantando, grabar antes la data antes que se pise
rango = 10 #la cantidad de iteraciones que voy a tener.
paso = 100 # cada cuanto es el paso entre una frecuencia y la siguiente
times = np.zeros((rango,2500)) #La cantidad de columnas es la cantidad de datos que me tira el osci
datas = np.zeros((rango,2500)) # creo que era 2500, esto habria que chequearlo
for i in range(0,rango):
    gen.freqs = i*paso #tiene el problema, de que el query ademas de hacer un write hace un read.
    osci.timebase = (1/(i*paso))/5  #Que cada 5 cuadraditos del Osci, halla 1 onda completa. Se vera si se ajusta
    time, data = osci.pantalla() #levanto pantalla
    xze, xin, yze, ymu, yoff = osci.escaleo2()#levanto el escaleo
    datas = np.array([float(j) for j in data]) #transformo en float y en array
    times = np.array([float(j) for j in time])
    datas = -yoff*ymu + datas*ymu #hago la conversion del escaleo
    times = xze + times * xin
    mdata[:][i] = datas #Primer valor es fila, el segundo es columna
    mtime[:][i] = times
np.savetxt('Nombre barrido freq data',mdata, delimiter=' ') 
np.savetxt('Nombre barrido freq tiempo',mtime, delimiter=' ') 

In [ ]:
rango = 10 #la cantidad de iteraciones que voy a tener.
paso = 100 # cada cuanto es el paso entre una frecuencia y la siguiente
times = np.zeros((rango,2500)) #La cantidad de columnas es la cantidad de datos que me tira el osci
datas = np.zeros((rango,2500)) # creo que era 2500, esto habria que chequearlo
for i in range(0,rango):
    gen.set_amp( i*paso)  #tiene el problema, de que el query ademas de hacer un write hace un read.
    osci.voltagescale = (1/(i*paso))/5  #Que cada 5 cuadraditos del Osci, halla 1 onda completa. Se vera si se ajusta
    time, data = osci.pantalla() #levanto pantalla
    xze, xin, yze, ymu, yoff = osci.escaleo2()#levanto el escaleo
    datas = np.array([float(j) for j in data]) #transformo en float y en array
    times = np.array([float(j) for j in time])
    datas = -yoff*ymu + datas*ymu #hago la conversion del escaleo
    times = xze + times * xin
    mdata[:][i] = datas #Primer valor es fila, el segundo es columna
    mtime[:][i] = times
np.savetxt('Nombre barrido amp data',mdata, delimiter=' ') 
np.savetxt('Nombre barrido amp tiempo',mtime, delimiter=' ') 